<h3> 구간 분할 </h3>

데이터 분석 알고리즘에 따라 연속 데이터보다 일정한 구간(bin)으로 나누어 분석하는 것이 효율적인 경우가 있다. 가격, 비용, 효율 등 연속적인 값을 일정한 수준이나 정도를 나타내는 이산적인 값으로 나타내어 구간별 차이를 드러내는 것이다. 이처럼 연속 변수를 일정한 구간으로 나누고, 각 구간을 범주형 이산 변수로 변화하는 과정을 **'구간 분할(binning)'**이라고 한다. <br>
판다스의 **cut() 함수**를 사용하면 구간 분할이 가능하다.<br>
먼저 'horsepower' 열의 값을 'float' 형으로 변환하고 numpy의 **histogram() 함수**를 활용해 구간을 나눠보자. **bins 매개변수**에 나누려는 구간의 개수를 입력한다. 각 구간에 속하는 값의 개수와 경계값 리스트를 반환한다. <br>
(4개의 경계값과 3개의 구간이 만들어진다.)

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('auto-mpg.csv', header = None)
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 

df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

count, bin_dividers = np.histogram(df['horsepower'], bins = 3)
print(count)
print('\n')
print(bin_dividers)

[257 103  32]


[ 46.         107.33333333 168.66666667 230.        ]


cut() 함수에는 **x 매개변수**에 범주형 자료로 변환할 열을 지정해야 한다. <br>
**bins 매개변수**에는 경계값 리스트를 전달하고, **labels 매개변수**에 각 구간의 이름을 지정한 리스트를 전달해야 한다. <br>
**include_lowest 매개변수**를 True로 지정하면 각 구간의 가장 낮은 경계값을 포함한다.

In [6]:
bin_names = ['저출력', '보통출력', '고출력']

df['hp_bin'] = pd.cut(x = df['horsepower'],
                      bins = bin_dividers,
                      labels = bin_names,
                      include_lowest = True)

df['hp_bin2'] = pd.cut(x = df['horsepower'],
                      bins = bin_dividers,
                      labels = bin_names,
                      include_lowest = False)

print(df[['horsepower', 'hp_bin', 'hp_bin2']].head(20))

    horsepower hp_bin hp_bin2
0        130.0   보통출력    보통출력
1        165.0   보통출력    보통출력
2        150.0   보통출력    보통출력
3        150.0   보통출력    보통출력
4        140.0   보통출력    보통출력
5        198.0    고출력     고출력
6        220.0    고출력     고출력
7        215.0    고출력     고출력
8        225.0    고출력     고출력
9        190.0    고출력     고출력
10       170.0    고출력     고출력
11       160.0   보통출력    보통출력
12       150.0   보통출력    보통출력
13       225.0    고출력     고출력
14        95.0    저출력     저출력
15        95.0    저출력     저출력
16        97.0    저출력     저출력
17        85.0    저출력     저출력
18        88.0    저출력     저출력
19        46.0    저출력     NaN


<h3> 더미 변수 </h3>

위와 같이 '저출력', '보통출력', '고출력'과 같은 문자열은 컴퓨터가 인식할 수 없다. 따라서 범주형 데이터를 회귀분석 등 머신러닝 알고리즘에 사용하려면 '컴퓨터가 인식 가능한 입력값'으로 변환해야 한다. <br>
이럴 때 숫자 0 또는 1로 표현되는 **'더미 변수(dummy variable)'**를 사용해야한다. 범주형 데이터를 컴퓨터가 인식할 수 있도록 숫자 0과 1로만 구성되는 **'원핫벡터(one_hot_vector)'**로 변환하는 것을 **'원핫인코딩(one_hot_encoding)'**이라고 부른다. <br>
판다스의 **get_dummies() 함수**를 사용하면 범주형 변수의 모든 고유값을 각각 새로운 더미 변수로 변환한다.

In [7]:
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


sklearn 라이브러리를 이용하면 원핫인코딩을 편하게 처리할 수 있다. <br>
데이터프레임 df의 'hp_bin'에 들어 있는 범주형 데이터를 0, 1을 원소로 갖는 원할벡터로 변환한다. 결과적으로 **희소행렬(sparse matrix)로 정리**된다. 희소행렬은 (행, 열) 좌표와 값 형태로 정리된다.

In [9]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()  # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()# one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환한다.
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))
print('\n')

# 2차원 행렬로 변환한다.
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))
print('\n')

# 희소행렬로 변환한다.
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>


[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>


  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
